Before you begin, execute this cell to import numpy and packages from the D-Wave Ocean suite, and all necessary functions for the gate-model framework you are going to use, whether that is the Forest SDK or Qiskit. In the case of Forest SDK, it also starts the qvm and quilc servers.

In [1]:
%run -i "assignment_helper.py"
%matplotlib inline
from pyquil.parameters import Parameter, quil_sin, quil_cos
from pyquil.quilbase import DefGate

Available frameworks:
Forest SDK
Qiskit
D-Wave Ocean


# State preparation



As explained in Maria Schuld's guest lecture, to prepare a state in a particular encoding is done by rotating vectors, in other words to map vectors into others, which is the basic idea of kernel learning. This procedure gives rise to interesting kernels, without the need of specifying the mapping function.


Let us work with an easy training data set of two vectors, $S = \{(\begin{bmatrix}0 \\ 1\end{bmatrix}, 0), (\begin{bmatrix}\frac{1}{2}\sqrt{2} \\ \frac{1}{2}\sqrt{2}\end{bmatrix}, 1)\}$. Let's have a test instance $\begin{bmatrix}1 \\ 0\end{bmatrix}$. We will build the interference circuit for this.

**Exercise 1** (1 point).

Create a circuit in your preferred framework that works on four qubits: ancilla, index, data, and class. Put the ancilla and index qubits into a uniform superposition. Verify for yourself that this also encodes the test data in the coefficients of the $|0000\rangle$, $|0100\rangle$ states as well as in the $|0010\rangle$, $| 0110\rangle$ states.

Insert identity operators for the data_qubit and the class_qubit, so these are initialized for amplitude measurements. 

Place your solution in an object called `circuit`. 

In [13]:
ancilla_qubit = 0
index_qubit = 1
data_qubit = 2
class_qubit = 3


###
### YOUR CODE HERE
###
'''
from pyquil import Program, get_qc
from pyquil.gates import *

training_set = [[0, 1], [1/np.sqrt(2), 1/np.sqrt(2)]]
labels = [0, 1]
test_set = [1, 0]

circuit = Program()
circuit += H(ancilla_qubit)
circuit += H(index_qubit)

circuit += X(ancilla_qubit)
circuit += CNOT(ancilla_qubit, data_qubit)
circuit += X(ancilla_qubit)

circuit += I(class_qubit)
'''
training_set = [[0, 1], [np.sqrt(2)/2, np.sqrt(2)/2]]
labels = [0, 1]
test_set = [[1, 0]]

test_angles = [2*np.arccos(test_set[0][0])/2]
training_angle = (2*np.arccos(training_set[1][0]))/4

angles = [test_angles[0], training_angle]

circuit = Program()
# Create uniform superpositions of the ancilla and index qubits
circuit += H(ancilla_qubit)
circuit += H(index_qubit)

# Apply Identity to Class state
circuit += I(data_qubit)
circuit += I(class_qubit)

get_amplitudes(circuit)


array([0.5+0.j, 0.5+0.j, 0.5+0.j, 0.5+0.j, 0. +0.j, 0. +0.j, 0. +0.j,
       0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
       0. +0.j, 0. +0.j])

In [14]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([0.5 +0.j, 0.5+0.j, 0.5 +0.j, 0.5+0.j, 0.+0.j, 0. +0.j, 0.+0.j,
       0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j]))

**Exercise 2** (1 point).

Extend the circuit to prepare the first training instance, by pushing the feature values into the coefficients of the $|0001\rangle$ and $|0101\rangle$, without affecting any of the other coefficients.

In [15]:
###
### YOUR CODE HERE
###
'''circuit += CCNOT(ancilla_qubit, index_qubit, data_qubit)
circuit += X(index_qubit)

get_amplitudes(circuit)
'''
circuit += X(index_qubit)
circuit += CCNOT(ancilla_qubit, index_qubit, data_qubit)
circuit += X(index_qubit)

In [16]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([0.5 +0.j, 0. +0.j, 0.5 +0.j, 0.5+0.j, 0.+0.j, 0.5+0.j, 0.+0.j,
       0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j]))

**Exercise 3** (2 points).

As a final step in the preparation, extend the circuit to push the feature values of the second training instance into the coefficients of the $|0011\rangle$ and $|0111\rangle$ states.

Remember that, as explained in the lecture notebook, loading vectors with amplitude encoding requires to apply a double controlled rotation by an angle determined by the coordinates of the vector.


In [25]:
circuit += CCNOT(ancilla_qubit, index_qubit, data_qubit)
circuit += CNOT(index_qubit, data_qubit)
circuit += H(data_qubit)

get_amplitudes(circuit)

array([0.707+0.j, 0.   +0.j, 0.   +0.j, 0.   +0.j, 0.707+0.j, 0.   +0.j,
       0.   +0.j, 0.   +0.j])

In [20]:
'''theta = Parameter('theta')
cry = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, quil_cos(theta / 2), -quil_sin(theta / 2)],
    [0, 0, quil_sin(theta / 2), quil_cos(theta / 2)]
])
cry_definition = DefGate('CRY', cry, [theta])
CRY = cry_definition.get_constructor()

# Double controlled Ry
alpha = Parameter('alpha')
ccry = np.array([
    [1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, quil_cos(alpha / 2), -quil_sin(alpha / 2)],
    [0, 0, 0, 0, 0, 0, quil_sin(alpha / 2), quil_cos(alpha / 2)]
])
ccry_definition = DefGate('CCRY', ccry,[alpha])
CCRY = ccry_definition.get_constructor()

'''

In [26]:
def get_angle(amplitude_0):
    return 2*np.arccos(amplitude_0)

In [27]:
circuit += X(ancilla_qubit)
circuit += H(data_qubit)
circuit += CCNOT(ancilla_qubit, index_qubit, data_qubit)
circuit += H(data_qubit)
circuit += X(ancilla_qubit)
get_amplitudes(circuit)

array([0.707+0.j, 0.   +0.j, 0.   +0.j, 0.   +0.j, 0.707+0.j, 0.   +0.j,
       0.   +0.j, 0.   +0.j])

In [28]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([0.5 +0.j,  0. +0.j,  0.5 +0.j,  np.sqrt(2)/4 + 0.j,  0.+0.j,  0.5+0.j,
       0.+0.j,  np.sqrt(2)/4 +0.j,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j, 0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j]))

ValueError: operands could not be broadcast together with shapes (8,) (16,) 

**Exercise 4** (1 point). 

Finish the state preparation circuit by a conditional flip of the class qubit, such that the coefficients of the second training vector and those of the copy of the test vector have the class bit equal to 1, without affecting any of the other coefficients.


It is very useful at this point, for testing purposes, to add code which shows the state vectors, as done in the lecture notebook


In [29]:
###
### YOUR CODE HERE
###
circuit += CNOT(index_qubit, class_qubit)

In [30]:
get_amplitudes(circuit)

array([0.707+0.j, 0.   +0.j, 0.   +0.j, 0.   +0.j, 0.707+0.j, 0.   +0.j,
       0.   +0.j, 0.   +0.j, 0.   +0.j, 0.   +0.j, 0.   +0.j, 0.   +0.j,
       0.   +0.j, 0.   +0.j, 0.   +0.j, 0.   +0.j])

In [31]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([0. +0.j, 0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j, 0. +0.j, 0. +0.j,
       0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0.5+0.j, 0. +0.j, 0. +0.j,
       0.5+0.j, 0. +0.j]))

AssertionError: 

In [32]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([ 0.5 +0.j,  0.  +0.j,  0. +0.j,  0.+0.j, 0.  +0.j,  0.5 +0.j,  0. +0.j,  0.+0., 0.  +0.j,  0.  +0.j,  0.5+0.j,  np.sqrt(2)/4 +0j, 0.  +0.j,  0.  +0.j,  0. +0.j,  np.sqrt(2)/4 +0.j]))

AssertionError: 

# Interference as a kernel

**Exercise 5** (1 point). 

At this point, our state is:

$|\psi\rangle = \frac{1}{\sqrt{2}}\sum_{m=0}^{1}|y_m\rangle|m\rangle|\psi_{x^m}\rangle|0\rangle + |y_m\rangle|m\rangle|\psi_{\tilde{x}}\rangle|1\rangle$

Apply the Hadamard gate on the ancilla to produce the interference and create: 

$ |\psi\rangle = \frac{1}{2\sqrt{2}}\sum_{m=0}^{1}|y_m\rangle|m\rangle(|\psi_{x^m}\rangle+|\psi_{\tilde{x}}\rangle)|0\rangle+|y_m\rangle|m\rangle(|\psi_{x^m}\rangle-|\psi_{\tilde{x}}\rangle)|1\rangle$


In [33]:
###
### YOUR CODE HERE
###
n_spins = 4
h = {v: 1 for v in range(n_spins)}
J = {(0, 1): -1, (1, 2): -1}
model = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.BINARY)

print(model.linear)
print(model.linear.values())
print(model.quadratic)
circuit += H(ancilla_qubit)

{0: 1, 1: 1, 2: 1, 3: 1}
ValuesView(<dimod.views.bqm.LinearView object at 0x7fabc847af08>)
{(0, 1): -1, (1, 2): -1}


In [34]:
get_amplitudes(circuit)

array([0.5+0.j, 0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j, 0.5+0.j, 0. +0.j,
       0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
       0. +0.j, 0. +0.j])

In [35]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([ 0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.707+0.j,
        0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,  0.   +0.j,
        0.354+0.j, -0.354+0.j,  0.   +0.j,  0.   +0.j, -0.354+0.j,
       -0.354+0.j]))

AssertionError: 

In [36]:
amplitudes = get_amplitudes(circuit)
assert np.allclose(amplitudes, np.array([  np.sqrt(2)/4+0.j , np.sqrt(2)/4+0.j , 0.        +0.j , 0.        +0.j,
  np.sqrt(2)/4+0.j ,-np.sqrt(2)/4+0.j , 0.        +0.j , 0.        +0.j,
  0.        +0.j,  0.        +0.j , (np.sqrt(2)+1)/4+0.j , (np.sqrt(2)-1)/4+0.j,
  0.        +0.j , 0.        +0.j , 0.25      +0.j ,-0.25      +0.j]))

AssertionError: 

If we measure the ancilla, the outcome probability of observing 0 will be $\frac{1}{4N}\sum_{i=1}^N |x_t + x_i|^2$. Performing post-selection on the 0 outcome, we can calculate the kernel and the probability of the test instance belonging to either class.